# QuakeMigrate example - Icequake detection at the Rutford icestream

## Overview

This notebook contains another example showing how to run QuakeMigrate for icequake detection.

Here, we detail how to:
1. Create travel-time lookup tables for the example seismometer network
2. Run the detect stage to coalesce energy through time
3. Run the trigger stage to determine events above a threshold value
4. Run the locate stage to refine the earthquake location

In [ ]:
import pandas as pd
from obspy.core import AttribDict
from pyproj import Proj

from QMigrate.io import Archive
from QMigrate.io import read_stations
from QMigrate.lut import compute, LUT
from QMigrate.signal import QuakeScan, Trigger
from QMigrate.signal.onset import STALTAOnset
from QMigrate.signal.pick import GaussianPicker

%matplotlib inline

In [ ]:
# --- i/o paths ---
station_file = "./inputs/rutford_stations.txt"
data_in = "./inputs/mSEED"
lut_out = "./outputs/lut/icequake.LUT"
run_path = "./outputs/runs"
run_name = "icequake_example"

## 1. Create travel-time lookup tables (LUTs)

In [ ]:
# --- Read in the station information file ---
stations = read_stations(station_file)

# --- Define the input and grid projections ---
gproj = Proj(proj="lcc", units="km", lon_0=-83.932, lat_0=-78.144, lat_1=-78.1,
             lat_2=-77.9, datum="WGS84", ellps="WGS84", no_defs=True)
cproj = Proj(proj="longlat", datum="WGS84", ellps="WGS84", no_defs=True)

# --- Define the grid specifications ---
# AttribDict behaves like a Python dict, but also has '.'-style access.
grid_spec = AttribDict()
grid_spec.ll_corner = [-84.14853353566141, -78.18825429331356, -0.350]
grid_spec.ur_corner = [-83.71921885073093, -78.10003166259442, 3.550]
grid_spec.cell_size = [0.1, 0.1, 0.1]
grid_spec.grid_proj = gproj
grid_spec.coord_proj = cproj

# --- Homogeneous LUT generation ---
lut = compute_traveltimes(grid_spec, stations, method="homogeneous", vp=3.841,
                          vs=1.970, log=True, save_file=lut_out)

## 2. Coalesce the seismic energy through time

In [ ]:
# --- Read in station file ---
stations = read_stations(station_file)

# --- Create new Archive and set path structure ---
archive = Archive(archive_path=data_in, stations=stations,
                  archive_format="YEAR/JD/*_STATION_*")

# --- Create new Onset ---
onset = STALTAOnset(position="classic")
onset.p_bp_filter = [20, 200, 4]
onset.s_bp_filter = [10, 125, 4]
onset.p_onset_win = [0.01, 0.25]
onset.s_onset_win = [0.05, 0.5]

# --- Create new QuakeScan ---
scan = QuakeScan(archive, lut, onset=onset, run_path=run_path,
                 run_name=run_name, log=True)

# --- Set detect parameters ---
scan.sampling_rate = 1000
scan.timestep = 0.75
scan.threads = 12

# --- Set time period over which to run detect ---
starttime = "2009-01-21T04:00:05.0"
endtime = "2009-01-21T04:00:15.0"

In [ ]:
# --- Run detect ---
scan.detect(starttime, endtime)

## 3. Run the trigger stage, to detect and output individual icequakes

nb: We can use the same QuakeScan object here because we are not using a different decimation. If running trigger and locate on grids with different levels of decimation, a new QuakeScan object should be created.

In [ ]:
# --- Create new Trigger ---
trig = Trigger(lut, run_path, run_name)

# --- Set trigger parameters ---
trig.marginal_window = 0.1
trig.minimum_repeat = 0.5
trig.normalise_coalescence = True

# --- Static threshold ---
trig.threshold_method = "static"
trig.static_threshold = 2.75

# --- Run trigger ---
trig.trigger(starttime, endtime, savefig=True)

## 4. Run the locate stage, to relocate triggered events on a less decimated grid

In [ ]:
# --- Create new PhasePicker ---
picker = GaussianPicker(onset=onset)
picker.marginal_window = 0.1
picker.plot_picks = True

# --- Create new QuakeScan ---
scan = QuakeScan(archive, lut, onset=onset, picker=picker,
                 run_path=run_path, run_name=run_name, log=True)

# --- Set locate parameters ---
scan.marginal_window = 0.1
scan.threads = 12
scan.sampling_rate = 1000

# Turn on plotting features
scan.plot_event_summary = True
scan.plot_event_video = False

# --- Toggle writing of waveforms ---
scan.write_cut_waveforms = False

In [ ]:
# --- Run locate ---
scan.locate(starttime=starttime, endtime=endtime)

## 5. Some of the key outputs

In [ ]:
# Show the .event file, containing event origin time and location:
icequake_event_fname = "./outputs/runs/icequake_example/locate/events/20090121040007151.event"
event_df = pd.read_csv(icequake_event_fname)

event_df

In [ ]:
# Show the .picks file, containing station time picks:
icequake_pick_fname = "outputs/runs/icequake_example/locate/picks/20090121040007151.picks"
pick_df = pd.read_csv(icequake_pick_fname)

pick_df

In [ ]:
# Show the coalescence pdf file, containing event origin time and location:
icequake_coal_image_fname = "outputs/runs/icequake_example/locate/summaries/icequake_example_20090121040007151_EventSummary.pdf"
from IPython.display import IFrame # For plotting pdf
IFrame(icequake_coal_image_fname, width=800, height=400) # Plot pdf